In [36]:
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
first_time = True
if first_time:
    nltk.download('stopwords')
    nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))
import html
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jotem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jotem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Questions

In [37]:
pth_questions = 'data\Questions.csv'
df_questions = pd.read_csv(pth_questions, encoding='latin1')
df_questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


### Answers 

In [38]:
pth_answers = 'data\Answers.csv'
df_answers = pd.read_csv(pth_answers, encoding='latin1')
df_answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


### Questions + Answers

In [39]:
merged_df = pd.merge(df_questions,df_answers, left_on='Id',  right_on='ParentId', how='inner')

In [41]:
col_names = {
    'Id_x': 'question_id',
    'Body_x': 'question_text', 
    'Id_y': 'answer_id', 
    'Body_y': 'answer_text', 
}

merged_df = merged_df.rename(columns=col_names, inplace=False)
#merged_df = merged_df.drop(columns = [ 'OwnerUserId_x', 'CreationDate_x', 'ClosedDate', 'Score_x', 'ParentId', 'OwnerUserId_y', 'CreationDate_y',  'Score_y'])
merged_df = merged_df[['question_id', 'Title', 'question_text', 'answer_text']]
merged_df = merged_df.drop_duplicates(subset=['question_id'], keep='first')

merged_df.head()
merged_df = merged_df.head(10000)

In [42]:
def clean_html_thorough(text):
    soup = BeautifulSoup(text, "html.parser")
    for element in soup(["script", "style", "img", "iframe", "noscript"]):
            element.decompose()
    return soup.get_text(separator=" ")

def clean_html(text):
    text_html_removed = BeautifulSoup(text, "lxml").text
    text_html_removed = html.unescape(text_html_removed)             # originally said "text_html_removed = html.unescape(text)" but should be html.unescape(text_html_removed)
    #text_html_removed = clean_html_thorough(text_html_removed)       # sim thing, chage back if I'm wrong though
    return text_html_removed

def tokenization(text):
    tokens = word_tokenize(text)
    tokens_cleaned = [word.lower() for word in tokens if word.lower() not in stop_words]
    return tokens_cleaned

merged_df['question_text'] = merged_df['question_text'].apply(clean_html)
merged_df['answer_text'] = merged_df['answer_text'].apply(clean_html)
merged_df['question_tokens'] = merged_df['question_text'].apply(tokenization)
merged_df['answer_tokens'] = merged_df['answer_text'].apply(tokenization)

merged_df.head()

,question_id,Title,question_text,answer_text,question_tokens,answer_tokens
0,80,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in S...,"I wound up using this. It is a kind of a hack,...","['ve, written, database, generation, script, s...","[wound, using, ., kind, hack, ,, actually, wor..."
3,90,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,Version Control with Subversion\nA very good r...,"[really, good, tutorials, explaining, branchin...","[version, control, subversion, good, resource,..."
6,120,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,The Jeff Prosise version from MSDN magazine wo...,"[anyone, got, experience, creating, sql-based,...","[jeff, prosise, version, msdn, magazine, works..."
7,180,Function for creating color wheels,This is something I've pseudo-solved many time...,I've read somewhere the human eye can't distin...,"[something, 've, pseudo-solved, many, times, n...","['ve, read, somewhere, human, eye, ca, n't, di..."
16,260,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,"Yes, I thought about that, but I soon figured ...","[little, game, written, c, #, ., uses, databas...","[yes, ,, thought, ,, soon, figured, another, d..."


In [45]:
data = merged_df.to_dict('records')
print(data[0:2])

[{'question_id': 80, 'Title': 'SQLStatement.execute() - multiple queries in one statement', 'question_text': "I've written a database generation script in SQL and want to execute it in my Adobe AIR application:\nCreate Table tRole (\n      roleID integer Primary Key\n      ,roleName varchar(40)\n);\nCreate Table tFile (\n    fileID integer Primary Key\n    ,fileName varchar(50)\n    ,fileDescription varchar(500)\n    ,thumbnailID integer\n    ,fileFormatID integer\n    ,categoryID integer\n    ,isFavorite boolean\n    ,dateAdded date\n    ,globalAccessCount integer\n    ,lastAccessTime date\n    ,downloadComplete boolean\n    ,isNew boolean\n    ,isSpotlight boolean\n    ,duration varchar(30)\n);\nCreate Table tCategory (\n    categoryID integer Primary Key\n    ,categoryName varchar(50)\n    ,parent_categoryID integer\n);\n...\n\nI execute this in Adobe AIR using the following methods:\npublic static function RunSqlFromFile(fileName:String):void {\n    var file:File = File.application